In [ ]:
from google.colab import drive, userdata
drive.mount("/content/drive")
HF_TOKEN = userdata.get("HF_CREDENTIALS")

Mounted at /content/drive


In [ ]:
!pip install datasets --quiet
!pip install emoji --quiet
!pip install evaluate --quiet
!pip install peft --quiet
!pip install accelerate --quiet
!pip install torch._inductor --upgrade --quiet
!pip install bitsandbytes --upgrade --quiet
from datasets import load_dataset, load_from_disk, Dataset, DatasetDict
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline, TrainingArguments, Trainer, DataCollatorWithPadding, RobertaTokenizer, RobertaForSequenceClassification
from peft import LoraConfig, get_peft_model, TaskType
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import os
import torch
import torch.nn.functional as F
import random
import numpy as np
import pandas as pd
import unicodedata
#from torch.utils.data import Dataset
import re
import emoji
import evaluate
import warnings
from sklearn.metrics import precision_recall_fscore_support, accuracy_score
from sklearn.model_selection import train_test_split

import torch._inductor.config
#torch._inductor.config.cpp.cxx_abi = "gnu"
torch.cuda.empty_cache()
warnings.filterwarnings("ignore")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 21.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 12.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-c

In [ ]:
# Settings
model_name = "microsoft/deberta-v3-base"
model_upload_name = "deBERTa_suicide_base"
lora_layers = ["query_proj", "key_proj", "value_proj"]
# ["query", "key", "value"] # BERT
# ["query_proj", "key_proj", "value_proj"] #Deberta
# ['Wi', 'Wqkv', 'Wo'] # ModernBERT
tokenized_dataset = "jordanfan/suicide_depression_tokenized_deBERTa_suicide_base"
batch_size = 16
epochs = 3
weight_decay=0.01
learning_rate = 2e-5

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name,
                                          token = HF_TOKEN
                                          )
#tokenizer.pad_token = tokenizer.eos_token
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
model = AutoModelForSequenceClassification.from_pretrained(model_name,
                                                           num_labels = 3,
                                                           token = HF_TOKEN,
                                                           ignore_mismatched_sizes=True
                                                           )
lora_config = LoraConfig(
    task_type = TaskType.SEQ_CLS,
    r=8,  # LoRA rank (lower values = less memory, faster training)
    lora_alpha=32,  # Scaling factor
    lora_dropout=0.1,  # Dropout for stability
    target_modules= lora_layers
)
model = get_peft_model(model, lora_config)

# Reinitialize the classification layer weights
torch.nn.init.xavier_uniform_(model.classifier.weight)

tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/579 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/371M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/371M [00:00<?, ?B/s]

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Parameter containing:
tensor([[-0.0113,  0.0317, -0.0621,  ...,  0.0423, -0.0834,  0.0681],
        [ 0.0287,  0.0737,  0.0796,  ..., -0.0074,  0.0595,  0.0442],
        [ 0.0173, -0.0422, -0.0427,  ...,  0.0812, -0.0709,  0.0768]],
       requires_grad=True)

In [ ]:
def preprocess_text(text):
    # Convert emojis to text
    text = emoji.demojize(text, delimiters=(" ", " "))
    # Remove URLs
    text = re.sub(r"http\S+|www\S+", "", text)
    # Remove user mentions (@username)
    text = re.sub(r"@\w+", "", text)
    # Remove excessive special characters
    text = re.sub(r"[^\w\s,.!?;:()]", "", text)
    # Normalize spacing
    text = text.strip()
    return text

def tokenize_batch(examples):
  return tokenizer(examples['text'], max_length = 512, return_tensors = "pt", padding = True, truncation = True)

def encode_labels(examples):
  return {"labels": label2id[examples["labels"]]}

if tokenized_dataset is None:
  df = load_dataset("jordanfan/suicide_depression")
  df = df.rename_column("class", "labels")
  train_df = df["train"].train_test_split(test_size = 0.2, seed = 42)
  df["train"] = train_df["train"]
  df["validation"] = train_df["test"]
  df = df.filter(lambda x: x["text"] is not None and x["labels"] is not None)\
        .filter(lambda x: len(x["text"]) > 10)
  df = df.map(lambda x: {"text": preprocess_text(x["text"])})
  df = df.map(tokenize_batch, batched=True)
  id2label = {0: "teenagers", 1: "depression", 2: "SuicideWatch"}
  label2id = {value: key for key, value in id2label.items()}
  df = df.map(encode_labels)
  df.push_to_hub(f"jordanfan/suicide_depression_tokenized_{model_upload_name}", private = True, token = HF_TOKEN)
else:
  df = load_dataset(tokenized_dataset, token = HF_TOKEN)

README.md:   0%|          | 0.00/667 [00:00<?, ?B/s]

train-00000-of-00002.parquet:   0%|          | 0.00/99.3M [00:00<?, ?B/s]

train-00001-of-00002.parquet:   0%|          | 0.00/98.7M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/61.7M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/49.7M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/222762 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/69613 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/55687 [00:00<?, ? examples/s]

In [ ]:
def compute_metrics(eval_pred):
    """Compute Accuracy, Precision, Recall, and F1-score."""
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    # Calculate metrics
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average="weighted")
    accuracy = accuracy_score(labels, predictions)
    # Compute Loss Manually
    logits_tensor = torch.tensor(logits)
    labels_tensor = torch.tensor(labels, dtype=torch.long)
    loss = F.cross_entropy(logits_tensor, labels_tensor).item()

    return {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1": f1,
        "loss": loss
    }

In [ ]:
args = TrainingArguments(
    output_dir=f"jordanfan/{model_upload_name}",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_dir="./logs",
    per_device_train_batch_size= batch_size,
    per_device_eval_batch_size= batch_size,
#    gradient_accumulation_steps=2,
    num_train_epochs=epochs,
    weight_decay=weight_decay,
    learning_rate = learning_rate,
    optim = "adamw_8bit",
    load_best_model_at_end=True,
    fp16=True,
    report_to=["tensorboard"],
    push_to_hub = True,
    hub_token = HF_TOKEN,
    hub_model_id = f"jordanfan/{model_upload_name}",
)

trainer = Trainer(
    model=model,
    args= args,
    train_dataset=df["train"],
    eval_dataset=df["validation"],
    tokenizer = tokenizer,
    compute_metrics=compute_metrics,
    data_collator=data_collator,
)

trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.346200,0.331995,0.863218,0.863479,0.863218,0.863057
2,0.320300,0.321777,0.869108,0.869246,0.869108,0.869123
3,0.307000,0.322459,0.868407,0.868751,0.868407,0.868527


No files have been modified since last commit. Skipping to prevent empty commit.


TrainOutput(global_step=41769, training_loss=0.3557095943992414, metrics={'train_runtime': 38096.1451, 'train_samples_per_second': 17.542, 'train_steps_per_second': 1.096, 'total_flos': 1.7675107332544512e+17, 'train_loss': 0.3557095943992414, 'epoch': 3.0})

In [ ]:
trainer.create_model_card(
    language='english',
    model_name=f"{model_upload_name}",
    tasks='classification',
    tags='classification',
    dataset='jordanfan/suicide_depression',
    dataset_args=f"Baseline"
)

commit_msg = f"Baseline"
trainer.push_to_hub(commit_message=commit_msg)

CommitInfo(commit_url='https://huggingface.co/jordanfan/deBERTa_suicide_base/commit/b4241c01895379473ce819da492ae28bed0f6f71', commit_message='Baseline', commit_description='', oid='b4241c01895379473ce819da492ae28bed0f6f71', pr_url=None, repo_url=RepoUrl('https://huggingface.co/jordanfan/deBERTa_suicide_base', endpoint='https://huggingface.co', repo_type='model', repo_id='jordanfan/deBERTa_suicide_base'), pr_revision=None, pr_num=None)